# 🍎 파이썬 머신러닝 완벽 가이드 혼공

### 2019.07.10 ~ 교재 8장

### 08. 텍스트 분석

## 07. 문서 군집화 소개와 실습(Opinion Review 데이터 세트) 

### 문서 군집화 개념 
비슷한 텍스트 구성의 문서를 군집화(clustering)하는 것이다. 텍스트 분류 기반의 문서 분류는 사전에 결정 카테고리 값을 가진 학습 데이터 세트가 필요한 데 반해, 문서 군집화는 학습 데이터 세트가 필요없는 **비지도학습** 기반으로 동작함. 

### Opinion Riview 데이터 세트를 이용한 문서 군집화 수행하기 
[UCI 머신러닝 리포지토리에 있는 Opinion Review 데이터 세트](https://archive.ics.uci.edu/ml/datasets/Opinosis+Opinion+%26frasl%3B+Review)를 사용. 

In [31]:
import pandas as pd 
import glob, os 
import re

path = r'/Users/harampark/Documents/superhotpot/CUAI/2020_1학기_머신러닝완벽가이드/data/OpinosisDataset1.0/topics'
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = []
opinion_text = []

# 개별 파일의 파일명은 filename_list로 취합, 
# 개별 파일의 파일 내용은 DataFrame 로딩 후 다시 string으로 변환해 opinion_text list로 취합 
for file_ in all_files: 
    # 개별 파일을 읽어 DataFrame으로 생성
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
    
    # 절대 경로로 주어진 파일면 제공. 
    # 맨 마지막 .data 확장자도 제거 
    filename_ = file_.split('\\')[-1]
    # print(filename_)
    filename = re.split('topics/|.txt.data', filename_)[1]  
    
    # 파일명 list와 파일 내용 list에 파일명과 파일 내용을 추가. 
    filename_list.append(filename)
    opinion_text.append(df.to_string())
    
# 파일명 list와 파일 내용 list 객체를 DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,battery-life_ipod_nano_8gb,short battery life I moved up from a...
1,gas_mileage_toyota_camry_2007,Ride seems comfortable and gas mileage fa...
2,room_holiday_inn_london,"We arrived at 23,30 hours and they could n..."
3,location_holiday_inn_london,Great location for tube and we crammed in...
4,staff_bestwestern_hotel_sfo,Staff are friendly and hel...


### TF-IDF 피처 벡터화 
TfidfVectorizer의 fit_transform()의 인자로 opinion_text 칼럼을 입력하면 개별 문서 텍스트에 대해 TF-IDF 변환된 피처 벡터화된 행렬을 구할 수 있음.

In [34]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer 

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                            ngram_range=(1,2), min_df=0.05, max_df=0.85)
#opinion_text 칼럼 값으로 피처 벡터화 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


### K-means 군집화 

문서별 텍스트가 TF-IDF 변환된 피처 벡터화 행렬 데이터에 대해 군집화를 수행해 어떤 문서끼리 군집되나 확인. 먼저 5개의 중심을 기반으로 군집화되는지 확인.

In [38]:
from sklearn.cluster import KMeans 

# 5개 집합으로 군집화 수행 
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [39]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,short battery life I moved up from a...,4
1,gas_mileage_toyota_camry_2007,Ride seems comfortable and gas mileage fa...,3
2,room_holiday_inn_london,"We arrived at 23,30 hours and they could n...",1
3,location_holiday_inn_london,Great location for tube and we crammed in...,1
4,staff_bestwestern_hotel_sfo,Staff are friendly and hel...,1


In [40]:
# 호텔 리뷰로 군집화
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
17,food_holiday_inn_london,The room was packed to capacity with queu...,0
32,food_swissotel_chicago,The food for our event was deli...,0
16,service_bestwestern_hotel_sfo,Both of us having worked in tourism for o...,0
27,service_holiday_inn_london,"not customer, oriented hotelvery low servi...",0
13,service_swissotel_hotel_chicago,Mediocre room and service for a very extr...,0


In [41]:
# 호텔 리뷰 군집화 2
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
31,bathroom_bestwestern_hotel_sfo,"The room was not overly big, but clean and...",1
49,free_bestwestern_hotel_sfo,The wine reception is a great idea as it i...,1
39,location_bestwestern_hotel_sfo,"Good Value good location , ideal ...",1
3,location_holiday_inn_london,Great location for tube and we crammed in...,1
50,parking_bestwestern_hotel_sfo,Parking was expensive but I think this is ...,1
28,price_holiday_inn_london,"All in all, a normal chain hotel on a nice...",1
2,room_holiday_inn_london,"We arrived at 23,30 hours and they could n...",1
46,rooms_bestwestern_hotel_sfo,"Great Location , Nice Rooms , Helpless...",1
30,rooms_swissotel_chicago,The Swissotel is one of our favorite hotel...,1
4,staff_bestwestern_hotel_sfo,Staff are friendly and hel...,1


In [42]:
# 차량용 네비게이션 리뷰 군집화
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
26,buttons_amazon_kindle,I thought it would be fitting to christen ...,2
34,directions_garmin_nuvi_255W_gps,You also get upscale features like spoken ...,2
48,display_garmin_nuvi_255W_gps,3 quot widescreen display was a ...,2
36,eyesight-issues_amazon_kindle,It feels as easy to read as the K1 but doe...,2
21,features_windows7,"I had to uninstall anti, virus and selecte...",2
44,fonts_amazon_kindle,Being able to change the font sizes is aw...,2
12,keyboard_netbook_1005ha,", I think the new keyboard rivals the gre...",2
38,navigation_amazon_kindle,"In fact, the entire navigation structure h...",2
41,price_amazon_kindle,"If a case was included, as with the Kindle...",2
10,satellite_garmin_nuvi_255W_gps,It's fast to acquire satel...,2


In [44]:
# 자동차 리뷰 군집화
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
18,comfort_honda_accord_2008,"Drivers seat not comfortable, the car its...",3
43,comfort_toyota_camry_2007,Ride seems comfortable and gas mileage fa...,3
1,gas_mileage_toyota_camry_2007,Ride seems comfortable and gas mileage fa...,3
45,interior_honda_accord_2008,I love the new body style and the interior...,3
22,interior_toyota_camry_2007,"First of all, the interior has way too ma...",3
35,mileage_honda_accord_2008,"It's quiet, get good gas mileage and look...",3
47,performance_honda_accord_2008,"Very happy with my 08 Accord, performance i...",3
42,quality_toyota_camry_2007,I previously owned a Toyota 4Runner which ...,3
29,seats_honda_accord_2008,Front seats are very uncomfor...,3
23,transmission_toyota_camry_2007,"After slowing down, transmission has to b...",3


In [45]:
# 전자기기 리뷰 군집화
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
33,accuracy_garmin_nuvi_255W_gps,", and is very, very acc...",4
9,battery-life_amazon_kindle,After I plugged it in to my USB hub on my ...,4
0,battery-life_ipod_nano_8gb,short battery life I moved up from a...,4
11,battery-life_netbook_1005ha,"6GHz 533FSB cpu, glossy display, 3, Cell 2...",4
15,performance_netbook_1005ha,The Eee Super Hybrid Engine utility lets u...,4
24,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,4
14,video_ipod_nano_8gb,"I bought the 8, gig Ipod Nano that has the...",4


그래서 군집화 개수를 줄여 군집화하자!

In [47]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화 
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_


# 소속 클러스터를 cluster_label 컬럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
45,interior_honda_accord_2008,I love the new body style and the interior...,0
22,interior_toyota_camry_2007,"First of all, the interior has way too ma...",0
23,transmission_toyota_camry_2007,"After slowing down, transmission has to b...",0
42,quality_toyota_camry_2007,I previously owned a Toyota 4Runner which ...,0
43,comfort_toyota_camry_2007,Ride seems comfortable and gas mileage fa...,0
35,mileage_honda_accord_2008,"It's quiet, get good gas mileage and look...",0
18,comfort_honda_accord_2008,"Drivers seat not comfortable, the car its...",0
29,seats_honda_accord_2008,Front seats are very uncomfor...,0
1,gas_mileage_toyota_camry_2007,Ride seems comfortable and gas mileage fa...,0
47,performance_honda_accord_2008,"Very happy with my 08 Accord, performance i...",0


### 군집별 핵심 단어 추출하기

KMeans 객체는 각 군집을 구성하는 단어 피처가 군집의 중심을 기준으로 얼마나 가깝게 위치해 있는지 cluster_centers_라는 속성으로 제공. 

In [48]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 2409)
[[0.         0.00137309 0.         ... 0.         0.         0.        ]
 [0.         0.00170335 0.0025537  ... 0.0032582  0.00349413 0.        ]
 [0.01819865 0.         0.         ... 0.         0.         0.00471073]]


군집이 3개, word 피처가 2409개라는 것. 각 행의 배열 값은 각 군집 내의 2409개 피처의 위치가 개별 중심과 얼마나 가까운가를 상대 값으로 나타낸 것. **0에서 1까지의 값을 가질 수 있으며 1에 가까울수록 중심과 가까운 값을 의미한다.** 

#### cluster_centers_ 속성값을 이용해 각군집별 핵심 단어 찾기 

- ndarray의 argsort()[:,::-1]를 이용하면 cluster_centers 배열 내 값이 큰 순으로 정렬된 위치 인덱스값 반환
- 위치 인덱스값이 필요한 이유는 핵심 단어 피처의 이름을 출력하기 위해서. 
- cluster_centers_ 배열 내에서 가장 값이 큰 데이터의 위치 인덱스를 추출한 뒤, 해당 인덱스를 이용해 핵심 단어 이름과 그 때의 상대위치 값을 추출해 cluster_details라는 Dict 객체 변수에 기록하고 반환하는 것이 get_cluster_details() 함수의 주요 로직

In [49]:
#군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함. 
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num,
                       top_n_features=10): 
    cluster_details = {}
    
    # cluster_centers array의 값이 큰 순으로 정렬된 인덱스 값 반환
    # 군집 중심점별 할당된 word 피처들이 거리값이 큰 순으로 값을 구하기 위함.
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    # 개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력 
    for cluster_num in range(clusters_num): 
        # 개별 군집별 정보를 담을 데이터 초기화 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num 
        
        # top n 피처 단어 구함
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함.
        top_feature_values = cluster_model.cluster_centers_[cluster_num, 
                                                           top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

In [50]:
# 더 보기좋게 보는 함수
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews 파일명 :',cluster_detail['filenames'][:7])
        print('==================================================')

In [51]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, 
                                     cluster_data=document_df, 
                                     feature_names=feature_names,
                                     clusters_num=3,
                                     top_n_features=10)
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'car', 'gas', 'transmission', 'gas mileage', 'ride', 'comfort']
Reviews 파일명 : ['gas_mileage_toyota_camry_2007', 'comfort_honda_accord_2008', 'interior_toyota_camry_2007', 'transmission_toyota_camry_2007', 'seats_honda_accord_2008', 'mileage_honda_accord_2008', 'quality_toyota_camry_2007']
####### Cluster 1
Top features: ['room', 'hotel', 'service', 'location', 'staff', 'food', 'clean', 'bathroom', 'parking', 'room wa']
Reviews 파일명 : ['room_holiday_inn_london', 'location_holiday_inn_london', 'staff_bestwestern_hotel_sfo', 'service_swissotel_hotel_chicago', 'service_bestwestern_hotel_sfo', 'food_holiday_inn_london', 'staff_swissotel_chicago']
####### Cluster 2
Top features: ['screen', 'battery', 'life', 'battery life', 'keyboard', 'kindle', 'size', 'button', 'easy', 'voice']
Reviews 파일명 : ['battery-life_ipod_nano_8gb', 'voice_garmin_nuvi_255W_gps', 'speed_garmin_nuvi_255W_gps', 'size_asus_netbook_1005ha', 'scr